
# Anomaly Detection - MOG


1. 관측 데이터가 다수의 가우시안 분포로부터 생성되었다고 가정하여 확률 밀도 함수 도출하기 

2. E / M 알고리즘을 통해서 객체 확률 / 가우시안 파라미터들의 값을 각각 최적화하기 

3. 적절한 가우시안 분포 개수 정하기 => class 밖에서 확인. 

**구현해야 하는 것**
- $g(x|\mu_m, \sum_m) = \frac{1}{(2\pi)^{d/2} |\sum_m|^{\frac{1}{2}}}exp(\frac{1}{2}(x-\mu_m)^T\sum_m^{-1}(x-\mu_m))$

> 또는 scipy.stats 의 multivariate_norm 모듈 채택 

- $p(x|\lambda) = \sum_{m=1}^Mw_mg(x|\mu_m, \sum_m)$

- 임의의 파라미터 값들을 부여하기 

- E Step : 가우시안 파라미터를 고정하여 객체 확률 값 계산 
> $p(m|x_i, \lambda) = \frac{w_mg(x_i|\mu_m, \sum_m)}{\sum_{k=1}^M w_mg(x_i|\mu_m, \sum_m)}$

- M Step : 객체 확률 값을 고정하여 가우시안 파라미터 값들 계산하기 

> $w_m^{new} = \frac{1}{N} \sum_{i=1}^N p(m|x_i, \lambda)$ 

> $\mu_m^{new} = \frac{\sum_{i=1}^N p(m|x_i, \lambda)x_i}{\sum_{i=1}^Np(m|x_i, \lambda)}$

> $\sigma_m^{2(new)} = \frac{\sum_{i=1}^N p(m|x_i, \lambda)x_i^2}{\sum_{i=1}^N p(m|x_i,\lambda)} - \mu_m^{2(new)}$



**필요한 것**
- X 
- d 
- m
- $\lambda = {w_m, \mu_m, \sum_M}$ 
- Var_type : Spherical / diagonal / full 
>  이건 이번에 고려 X. 기본적으로 Diagonal을 가정 



**함수의 형태**
- def __init__(self, X, Var_type) : 
  
- def lambda(self) : => m개의 [w_1, $\mu_1, \sum_1$] list를 포괄한 list제작 
> 분산의 형태에 대해서 지정해줄 수 있어야 함. 

- def expectation(self) : self.lambda를 입력값으로 받아 self.p 값 업데이트 
- def maximization(self) : self.p 를 입력값으로 받아 self.lambda 값 업데이트 

- def sol(self) : E/M 과정의 답이 오차 내로 줄어들때까지 진행. 최후 우도값으로 반환할 수 있을 것. 


In [4]:
import numpy as np
import pandas as pd
import random as rand

from sklearn.datasets import load_iris
X = load_iris()['data']

import matplotlib.pyplot as plt
import scipy as sp
from scipy.stats import multivariate_normal

from scipy.stats import norm
from sys import maxsize

In [3]:
class MOG() : 
    def __init__(self, X, m) : 
        self.X = np.array(X) 
        self.n = np.shape(X)[0] 
        self.d = np.shape(X)[1]
        
        self.m = m 
        
        self.w, self.mu, self.sigma, self.cov = self.set_parameter()
        self.p = self.expectation()
        
    def set_parameter(self) : 
        # w 는 m분의 1로 균등하게 부여 
        w = np.ones(self.m) / self.m 
        
        # mu는 각 변수들의 최소 ~ 최대 값 사이의 랜덤한 값으로 부여 
        min_value = np.apply_along_axis(lambda a : np.min(a), 0, self.X)
        max_value = np.apply_along_axis(lambda a : np.max(a), 0, self.X)
        mu = [] 
        sigma = [] 
        for i in range(self.m) : 
            np.random.seed(i)
            mu.append([np.random.uniform(max_value[j], min_value[j]) for j in range(self.d)])
            sigma.append([np.random.uniform(max_value[j] - mu[i][j], mu[i][j] -min_value[j]) for j in range(self.d)])    
        mu = np.array(mu)
        sigma = np.array(sigma)
        cov = [np.diag(sigma[i]) for i in range(self.m)] 
        
        return w, mu, sigma, cov

    
    def g(self, x, m) : 
        """
        x = np.array(x)
        inv_cov = np.linalg.inv(self.cov[m])
        upper_value =  np.exp((x-self.mu[m])@inv_cov@(x-self.mu[m]).T /2)
        under_value = ((2*np.pi)**(self.d/2)) * np.sqrt(np.sqrt((self.cov[m]**2).sum())) 
        print(upper_value)
        print(under_value)
        """     
        # g의 계산을 Multivariate_normal.pdf 를 구현했을 시 원하는 답이 나옴. 
        p = multivariate_normal.pdf(x, mean = self.mu[m], cov=self.cov[m])
  
        return p

    def expectation(self): 
        p = np.zeros((self.n, self.m))
        
        for i in range(self.n) : 
            vector = np.array([self.g(self.X[i], j) for j in range(self.m)]) 
            p[i] = vector / vector.sum()
        return p # m x n 행렬 
    
    def maximization(self) :
        w = self.p.sum(axis= 0) / self.n
        mu = np.array([(self.X * self.p[:,j].reshape(-1,1)).sum(axis=0) / self.p[:,j].sum(axis=0) for j in range(self.m) ]) 
        sigma = np.array([(self.X**2 * self.p[:,j].reshape(-1,1)).sum(axis=0) / self.p[:,j].sum(axis=0) - mu[j]**2 for j in range(self.m) ]) 
        cov = np.array([np.diag(sigma[i]) for i in range(self.m)]) 
        return w, mu, sigma, cov 
        
    def find_parameter(self, epsilon = 1e-40) : 
        pre = self.p[:,:] 
        self.p = self.expectation() 
        self.w, self.mu, self.sigma, self.cov = self.maximization() 
        diff = pre - self.p
        while diff.sum() > epsilon : 
            pre = self.p[:,:]
            self.p = self.expectation() 
            self.w, self.mu, self.sigma, self.cov = self.maximization()
            diff = pre - self.p 
        
        return self.p, self.w, self.mu, self.sigma, self.cov
    
    def check_abnormal(self,x, epsilon = 1e-5) : 
        value = np.array([self.w[i] * self.g(x,i) for i in range(self.m)]).sum()
        if value < epsilon : 
            return print("abnormal data. p:" ,value)
        else : 
            return print("normal data. p:", value)
        

In [22]:
test = MOG(X, 3)
test.check_abnormal(a)
test.check_abnormal(b)
test.check_abnormal(c)
test.check_abnormal([0,0,0,10])


normal data. p: 0.0010460427740777726
normal data. p: 0.0009746061652816432
normal data. p: 0.0007911697689929761
abnormal data. p: 9.121430279286356e-25


In [18]:
a = X[0]
b= X[1]
c = X[2]